In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import glob
import subprocess

In [3]:
def call_RNAfold(sequence):
    MyOut = subprocess.Popen(['RNAfold', '-p', '--noPS', '--constraint'],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT)
    stdout, stderr = MyOut.communicate(input=str.encode(sequence))
    return stdout

def get_energy_RNAfold(stdout_string):
    temp = stdout_string.decode("utf-8") 
    energy_line = temp.split('\n')[-5]
    energy_val = energy_line[energy_line.index(' '):]
    energy_val = energy_val.strip().strip('()').strip()
    
    mfe_line = temp.split('\n')[-4]
    mfe_val = mfe_line[energy_line.index(' '):]
    mfe_val = mfe_val.strip().strip('[]').strip()
    return float(energy_val), float(mfe_val)    

In [4]:
for host_tsv_file in glob.glob('../Data/host_genomes/' + '*.clean.tsv'):
    if '562.clean.tsv' not in host_tsv_file:
        continue
    print(host_tsv_file)
    df = pd.read_csv(host_tsv_file, sep = "\t", index_col = 0)
    for index in df.index[:]:
        us_seq = df.at[index, 'upstream_sequence'] 
        cds_seq = df.at[index, 'coding_sequence'][:30]
        if len(us_seq) == 30 and len(cds_seq) == 30:
            rna_out = call_RNAfold(us_seq+cds_seq)
            e1, e2 = get_energy_RNAfold(rna_out)
            df.at[index, 'secondary_structure'] = e2
    
    df.to_csv(host_tsv_file, sep = "\t")

../Data/host_genomes/562.clean.tsv


In [6]:
for virus_folder in glob.glob('../Data/*_rep_viruses/'):
    if '562_rep_viruses' not in virus_folder:
        continue
    print('###', virus_folder)
    for virus_tsv_file in glob.glob(virus_folder + '*.clean.tsv'):
        print(virus_tsv_file)
        ###
        df = pd.read_csv(virus_tsv_file, sep = "\t", index_col = 0)
        for index in df.index[:]:
            us_seq = df.at[index, 'upstream_sequence'] 
            cds_seq = df.at[index, 'coding_sequence'][:30]
            if len(us_seq) == 30 and len(cds_seq) == 30:
                rna_out = call_RNAfold(us_seq+cds_seq)
                e1, e2 = get_energy_RNAfold(rna_out)
                df.at[index, 'secondary_structure'] = e2
        df.to_csv(virus_tsv_file, sep = "\t")

### ../Data/562_rep_viruses/
../Data/562_rep_viruses/6088.clean.tsv
../Data/562_rep_viruses/6277.clean.tsv
../Data/562_rep_viruses/2454.clean.tsv
../Data/562_rep_viruses/3758.clean.tsv
../Data/562_rep_viruses/6382.clean.tsv
../Data/562_rep_viruses/197.clean.tsv
../Data/562_rep_viruses/2797.clean.tsv
../Data/562_rep_viruses/710.clean.tsv
../Data/562_rep_viruses/3861.clean.tsv
../Data/562_rep_viruses/2440.clean.tsv
../Data/562_rep_viruses/12799.clean.tsv
../Data/562_rep_viruses/4785.clean.tsv
../Data/562_rep_viruses/5461.clean.tsv
../Data/562_rep_viruses/3071.clean.tsv
../Data/562_rep_viruses/6462.clean.tsv
../Data/562_rep_viruses/3247.clean.tsv
../Data/562_rep_viruses/5077.clean.tsv
../Data/562_rep_viruses/6903.clean.tsv
../Data/562_rep_viruses/5852.clean.tsv
../Data/562_rep_viruses/12529.clean.tsv
../Data/562_rep_viruses/5753.clean.tsv
../Data/562_rep_viruses/1036.clean.tsv
../Data/562_rep_viruses/635.clean.tsv
../Data/562_rep_viruses/3033.clean.tsv
../Data/562_rep_viruses/4755.clean.t

../Data/562_rep_viruses/4406.clean.tsv
../Data/562_rep_viruses/1223.clean.tsv
../Data/562_rep_viruses/3156.clean.tsv
../Data/562_rep_viruses/3540.clean.tsv
../Data/562_rep_viruses/170.clean.tsv
../Data/562_rep_viruses/7680.clean.tsv
../Data/562_rep_viruses/401.clean.tsv
../Data/562_rep_viruses/637.clean.tsv
../Data/562_rep_viruses/11438.clean.tsv
../Data/562_rep_viruses/2940.clean.tsv
../Data/562_rep_viruses/18613.clean.tsv
../Data/562_rep_viruses/2745.clean.tsv
../Data/562_rep_viruses/1399.clean.tsv
../Data/562_rep_viruses/4928.clean.tsv
../Data/562_rep_viruses/6386.clean.tsv
../Data/562_rep_viruses/2982.clean.tsv
../Data/562_rep_viruses/12576.clean.tsv
../Data/562_rep_viruses/7934.clean.tsv
../Data/562_rep_viruses/14172.clean.tsv
../Data/562_rep_viruses/7057.clean.tsv
../Data/562_rep_viruses/2996.clean.tsv
../Data/562_rep_viruses/3748.clean.tsv
../Data/562_rep_viruses/652.clean.tsv
../Data/562_rep_viruses/6501.clean.tsv
../Data/562_rep_viruses/9340.clean.tsv
../Data/562_rep_viruses/2

### Double checking sequence extraction

In [35]:
from Bio import SeqIO

In [40]:
taxa = 287
tsv_loc = '../Data/host_genomes/{}.clean.tsv'.format(taxa)
df = pd.read_csv(tsv_loc, sep = "\t", index_col = 0)

fasta_loc = '../Data/host_genomes/{}.fasta'.format(taxa)
genome_seq = SeqIO.read(fasta_loc, 'fasta')

gff3_loc = '../Data/host_genomes/{}.gff3'.format(taxa)
gff3_df = pd.read_csv(gff3_loc, sep='\t', comment='#', header=None)

In [41]:
seq_dicty = {}
gff3_df = gff3_df[gff3_df[2]=='CDS']

for index in gff3_df.index:
    if gff3_df.at[index, 6] == '+':
        start = gff3_df.at[index, 3]
        locus_tag = gff3_df.at[index, 8].split(';Parent=gene-')[1].split(';')[0]
        seq_dicty[locus_tag] = genome_seq.seq[start-31:start+29]
for index in df.index:
    if df.at[index, 'strand'] == '+':
        assert df.at[index, 'upstream_sequence']+df.at[index, 'coding_sequence'][:30] ==\
                seq_dicty[df.at[index, 'locus_tag']]

In [42]:
seq_dicty = {}
gff3_df = gff3_df[gff3_df[2]=='CDS']

for index in gff3_df.index:
    if gff3_df.at[index, 6] == '-':
        start = gff3_df.at[index, 4]
        locus_tag = gff3_df.at[index, 8].split(';Parent=gene-')[1].split(';')[0]
        seq_dicty[locus_tag] = genome_seq.seq[start-30:start+30].reverse_complement()
for index in df.index:
    if df.at[index, 'strand'] == '-':
        assert df.at[index, 'upstream_sequence']+df.at[index, 'coding_sequence'][:30] ==\
                seq_dicty[df.at[index, 'locus_tag']]